In [ ]:
# env variables: AZURE_CLIENT_ID, AZURE_CLIENT_ID, AZURE_TENANT_ID, RESOURCE_GROUP, SUBSCRIPTION_ID

# url example: 'https://<storage_account_name>.blob.core.windows.net/<container_name>/<path>'

# aml url example: 'azureml://subscriptions/<subscription_id>/resourcegroups/<resource_group_name>/workspaces/<workspace_name>/datastores/<datastore_name>/paths/<path>'

# image_urls example: ['url', 'url']
# Labels example: [{'task1': url, 'task2': 'url'}, {'task1': url, 'task3': 'url', 'task2': 'url'}]
# tags example: [{'tag1': 'value1:string', 'tag2': 'value2:string'}, {'tag1': 'value1:string', 'tag2': 'value2:string'}]

In [1]:
import MLtableBuilder as mb

In [2]:
# reload the module if it has been modified
import importlib
importlib.reload(mb)

<module 'MLtableBuilder' from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alibina8/code/Users/alibina/Mlops-cli2-classic/data-science/experiment/bosch/data_managment/MLtableBuilder/MLtableBuilder.py'>

In [3]:
# add clinent id, client secret, tenant id, aml workspace, resource group, subscription id to the environment variables
import os
# os.environ["AZURE_CLIENT_ID"] = ""
# os.environ["AZURE_CLIENT_SECRET"] = ""
# os.environ["AZURE_TENANT_ID"] = ""
# read .env file as environment variables
with open("../../../.env", "r") as f:
    for line in f:
        if line.strip() != "":
            key, value = line.strip().split("=", 1)
            os.environ[key] = value

In [4]:
# read csv file
import pandas as pd
# df = pd.read_csv("./validation_annotations.csv")
# df.columns = ["image_url_aml", "classifications"]
# df['label'] = "validation"
df = pd.read_csv("./validation_annotations_absoult_path.csv")
df.head()

,image_url_aml,classifications,label,image_url_blob,storage_account_name,container_name,path
0,azureml://subscriptions/f804f2da-c27b-45ac-bf8...,can,validation,https://stmlopsv2clas2.blob.core.windows.net/c...,stmlopsv2clas2,stacccontainera,UI/2023-02-01_192745_UTC/can/1.jpg
1,azureml://subscriptions/f804f2da-c27b-45ac-bf8...,can,validation,https://stmlopsv2clas2.blob.core.windows.net/c...,stmlopsv2clas2,stacccontainera,UI/2023-02-01_192745_UTC/can/14.jpg
2,azureml://subscriptions/f804f2da-c27b-45ac-bf8...,can,validation,https://stmlopsv2clas2.blob.core.windows.net/c...,stmlopsv2clas2,stacccontainera,UI/2023-02-01_192745_UTC/can/19.jpg
3,azureml://subscriptions/f804f2da-c27b-45ac-bf8...,can,validation,https://stmlopsv2clas2.blob.core.windows.net/c...,stmlopsv2clas2,stacccontainera,UI/2023-02-01_192745_UTC/can/23.jpg
4,azureml://subscriptions/f804f2da-c27b-45ac-bf8...,can,validation,https://stmlopsv2clas2.blob.core.windows.net/c...,stmlopsv2clas2,stacccontainera,UI/2023-02-01_192745_UTC/can/28.jpg


In [5]:
# # 'azureml://subscriptions/<subscription_name>/resourcegroups/<resource_group_name>/workspaces/<workspace_name>/datastores/<datastore_name>/paths/<path>
# azureml://subscriptions/f804f2da-c27b-45ac-bf80-16d4d331776d/resourcegroups/rg-mlopsv2clas-505prod/workspaces/mlw-mlopsv2clas-505prod/datastores/stacccontainera/paths/UI/2023-02-01_192745_UTC/can/1.jpg
# azureml://subscriptions/f804f2da-c27b-45ac-bf80-16d4d331776d/resourcegroups/rg-mlopsv2clas-505prod/workspaces/mlw-mlopsv2clas-505prod/datastores/stmlopsv2clas2_containera/paths/1.jpg
# https://<storage_account_name>.blob.core.windows.net/<container_name>/<path>

# # #  create a dictionary of for each image containing the datastore_name, path, and the image url
# image_urls = []
# storage_account_names = []
# paths = []
# container_names = []
# for i in range(len(df)):
#     image_url_aml = df["image_url_aml"][i]
#     # get the path
#     path = image_url_aml.split("paths/", 1)[-1]
#     # get the datastore name
#     datastore_name = image_url_aml.split("datastores/", 1)[1].split("/paths/", 1)[0]
#     ds = table.aml_client.datastores.get(datastore_name)
#     storage_account_name = ds.account_name
#     container_name = ds.container_name
#     image_url = f"https://{storage_account_name}.blob.core.windows.net/{container_name}/{path}"
#     image_urls.append(image_url)
#     storage_account_names.append(storage_account_name)
#     container_names.append(datastore_name)
#     paths.append(path)

# df['image_url_blob'] = image_urls
# df['storage_account_name'] = storage_account_names
# df['container_name'] = container_names
# df['path'] = paths
# df.head()
# # remove the image_url_aml column
# # df = df.drop(columns=["image_url_aml"])
# # save to csv file
# df.to_csv("./validation_annotations_absoult_path.csv", index=False)
# df.head()

In [6]:
tasklist = ["classifications"]
taglist = ["label"]
table = mb.MLtableBuilder(table_name="validation_annotations_absoult_path",
                          table_version=6,
                          image_urls=df.image_url_blob.tolist(), 
                          Labels=[{col: df[col][i] for col in df.columns if col in tasklist} for i in range(len(df))],
                          tags=[{col: df[col][i] for col in df.columns if col in taglist} for i in range(len(df))],
                          aml_workspace="mlw-mlopsv2clas-505prod")

In [7]:
table.check_table_name()

True

In [8]:
table.create_jsonl()

The jsonl file is created: ./mltable/validation_annotations_absoult_path.jsonl


True

In [9]:
table.create_ml_table_file()

In [10]:
table.create_aml_table()

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'validation_annotations_absoult_path', 'description': 'MLTable dataset', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alibina8/code/Users/alibina/Mlops-cli2-classic/data-science/experiment/bosch/data_managment/MLtableBuilder', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f022165e170>, 'version': '6', 'latest_version': None, 'path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/alibina8/code/Users/alibina/Mlops-cli2-classic/data-science/experiment/bosch/data_managment/MLtableBuilder/mltable'), 'datastore': None})

In [11]:
table.upload_table()

The jsonl file is created: ./mltable/validation_annotations_absoult_path.jsonl


Uploading mltable (0.03 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29022/29022 [00:00<00:00, 709853.46it/s]
